In [ ]:
import xmltodict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from datetime import datetime

pd.options.display.max_rows = 99

arquivo_add = str(datetime.now())[-6:]


# Malha

In [ ]:
with open('malha.net.xml', 'rb') as f:
    malha_xml = xmltodict.parse(
        f, attr_prefix='', cdata_key='', dict_constructor=dict)


In [ ]:
malha_viaria = []

for attr in malha_xml['net']['edge']:
    malha_viaria.append([attr['id'], attr['name'], attr['priority'],
                         attr['param'][0]['value'], attr['param'][1]['value']])

malha = pd.DataFrame(malha_viaria, columns=[
                     'id', 'name', 'priority', 'vmax', 'vmin'])

malha.sort_values('priority', inplace=True, ascending=False)
malha[['priority', 'vmax', 'vmin']] = malha[[
    'priority', 'vmax', 'vmin']].apply(pd.to_numeric)

malha.head()


# Rotas e demandas

In [ ]:
with open('demanda.rou.xml', 'rb') as f:
    dem_xml = xmltodict.parse(
        f, attr_prefix='', cdata_key='', dict_constructor=dict)

# print(json.dumps(dem_xml['routes']['route'], indent=4, ensure_ascii=False))

In [ ]:
rotas = pd.DataFrame(dem_xml['routes']['route'])

rotas.drop(columns=['color'], inplace=True)

rotas.head()

In [ ]:
demandas = pd.DataFrame(dem_xml['routes']['flow'])

demandas.head(25)

In [ ]:
demandas.loc[demandas['route'] == 'route_0']['number'].values[0]

# Resultados da simulação

In [ ]:
result_sumo = pd.read_csv('full.output.csv', sep=';')

In [ ]:
result_sumo['vehicle_edge'] = result_sumo['vehicle_lane'].str.split(
    '_').str[0]

veics_sumo = result_sumo[result_sumo['vehicle_route'].notnull(
)][['data_timestep', 'vehicle_route', 'vehicle_edge', 'vehicle_speed']]

veic_sumo = veics_sumo.groupby(['vehicle_route', 'vehicle_edge'], as_index=False)[
    'vehicle_speed'].median()

veic_sumo.head()


# Detectores

In [ ]:
tree_add = ET.parse('adicionais.xml')
root_add = tree_add.getroot()

b2tf_add = root_add.findall('./e1Detector')

addicional_atual = b2tf_add[0].attrib['file']

with open(addicional_atual, 'rb') as f:
    detect = xmltodict.parse(
        f, attr_prefix='', cdata_key='', dict_constructor=dict)

In [ ]:
detec_list = []

for attr in detect['detector']['interval']:
    detec_list.append(
        [attr['id'], float(attr['speed']), int(attr['nVehEntered'])])

detectores = pd.DataFrame(detec_list, columns=[
    'name', 'speed', 'nVehEntered'])

detectores['id'] = detectores['name'].str.split('_').str[1]


In [ ]:
detectores_gb = detectores.groupby(['id']).agg({'speed':'median', 'nVehEntered':'sum'}).reset_index()

In [ ]:
detectores_malha = pd.merge(
    detectores_gb, malha, how='left', on=['id']).reset_index()
result_sumo.head()

In [ ]:
def comparar_vels(x):
    if x['speed'] > x['vmax']:
        return x['speed'] - x['vmax']
    elif x['speed'] < x['vmin']:
        return x['speed'] - x['vmin']
    else:
        return 0

detectores_malha['dif'] = detectores_malha.apply(comparar_vels, axis=1)


In [ ]:
detectores_malha.head(25)

# resultados por rota

In [ ]:
rotas_det = {}
detect_edges = set(detectores_malha['id'])

for idx, row in rotas.iterrows():
    rotas_det[row['id']] = {
        'priority': malha.loc[malha['id'] == row['edges'].split()[0]]['priority'].values[0],
        'edge': [],
        'vmax': [],
        'vmin': [],
        'sumo': [],
        'demanda_atrib': None,
        'sumo_sensor_speed1': None,
        'sumo_sensor_speed2': None,
        'vmax_sensor1': None,
        'vmax_sensor2': None,
        'vmin_sensor1': None,
        'vmin_sensor2': None,
        'sensor_edge1': None,
        'sensor_edge2': None,
        'sensor_speed1': None,
        'sensor_speed2': None,
        'sensor_veic1':  None,
        'sensor_veic2':  None,
        'demand_num': None
    }

    demanda_xml = int(
        demandas.loc[demandas['route'] == row['id']]['number'].values[0])
    print(demanda_xml)

    rotas_det[row['id']]['demanda_atrib'] = demanda_xml
    rotas_det[row['id']]['demand_num'] = demanda_xml

    for edge in row['edges'].split():
        rotas_det[row['id']]['edge'].append(edge)
        vmax = malha[malha['id'] == edge]['vmax'].values[0]
        rotas_det[row['id']]['vmax'].append(vmax)
        vmin = malha[malha['id'] == edge]['vmin'].values[0]
        rotas_det[row['id']]['vmin'].append(vmin)
        speed_sumo = veic_sumo[veic_sumo['vehicle_edge']
                               == edge]['vehicle_speed'].values[0]
        rotas_det[row['id']]['sumo'].append(speed_sumo)

        if edge in detect_edges:
            detector_edge = detectores_malha[detectores_malha['id']
                                             == edge]['id'].values[0]

            if edge == detector_edge and rotas_det[row['id']]['sensor_edge1'] == None:
                rotas_det[row['id']]['sensor_speed1'] = detectores_malha[detectores_malha['id']
                                                                         == edge]['speed'].values[0]
                rotas_det[row['id']]['sensor_veic1'] = detectores_malha[detectores_malha['id']
                                                                        == edge]['nVehEntered'].values[0]
                rotas_det[row['id']]['sensor_edge1'] = detector_edge

                rotas_det[row['id']]['sumo_sensor_speed1'] = speed_sumo
                rotas_det[row['id']]['vmax_sensor1'] = vmax
                rotas_det[row['id']]['vmin_sensor1'] = vmin

            elif edge == detector_edge:
                rotas_det[row['id']]['sensor_speed2'] = detectores_malha[detectores_malha['id']
                                                                         == edge]['speed'].values[0]
                rotas_det[row['id']]['sensor_veic2'] = detectores_malha[detectores_malha['id']
                                                                        == edge]['nVehEntered'].values[0]
                rotas_det[row['id']]['sendor_edge2'] = detector_edge

                rotas_det[row['id']]['sumo_sensor_speed2'] = speed_sumo
                rotas_det[row['id']]['vmax_sensor2'] = vmax
                rotas_det[row['id']]['vmin_sensor2'] = vmin


In [ ]:
rotas_det.keys()

In [ ]:
rotas_det['route_14']

## Gráfico com a progressão da rota na simulação / velocidade máxima e mínima esperada.

In [ ]:
rota_plot = pd.DataFrame.from_dict(rotas_det['route_12'])
rota_plot_label1 = f"vel.sensor edge: {rota_plot['sensor_edge1'].values[0]}"
rota_plot_label2 = f"vel.sensor edge: {rota_plot['sensor_edge2'].values[0]}"
col = rota_plot.loc[:, 'vmax':'vmin']
plt.plot('edge', 'sumo', label='vel.sumo', c='red', data=rota_plot)
plt.plot('edge', 'sensor_speed1', label=rota_plot_label1,
         c='black', linestyle="--", data=rota_plot)
plt.plot('edge', 'sensor_speed2', label=rota_plot_label2,
         c='green', linestyle="--", data=rota_plot)
plt.fill_between(x='edge', y1='vmin', y2='vmax', data=rota_plot)
plt.legend()
plt.show()


In [ ]:
fig = plt.figure(figsize=(30, 25))
fig.subplots_adjust(hspace=.4, wspace=.1)

for idx, rota in enumerate(rotas_det.keys()):
    rota_plot = pd.DataFrame.from_dict(rotas_det[rota])
    rota_plot_label1 = f"vel.sensor edge: {rota_plot['sensor_edge1'].values[0]}"
    rota_plot_label2 = f"vel.sensor edge: {rota_plot['sensor_edge2'].values[0]}"
    fig.add_subplot(6, 4, idx+1)
    plt.plot('edge', 'sumo', label='vel.sumo', c='red', data=rota_plot)
    plt.plot('edge', 'sensor_speed1', label=rota_plot_label1,
             c='black', linestyle="--", data=rota_plot)
    plt.plot('edge', 'sensor_speed2', label=rota_plot_label2,
             c='green', linestyle="--", data=rota_plot)
    #rota_plot.plot(x='edge', y='sumo', c='red', ax=ax)
    plt.fill_between(x='edge', y1='vmin', y2='vmax', data=rota_plot)
    plt.legend(loc=4)
    plt.title(rota)

# plt.savefig(f'resultados/plots/{arquivo_add}.png')
plt.show


## novos valores rota, prioridade 4 (máxima encontrada no início de uma rota)

Alterações no número de veículos próxima simulação

In [ ]:
malha['priority'].unique()

In [ ]:
for chave, valor in rotas_det.items():
    sumo_sensor_speed1 = valor['sumo_sensor_speed1']
    vmax_sensor1 = valor['vmax_sensor1']
    sumo_sensor_speed2 = valor['sumo_sensor_speed2']
    vmax_sensor2 = valor['vmax_sensor2']
    demanda_atrib = valor['demanda_atrib']
    print(
        f'route:{chave}, prioridade:{valor["priority"]}, sumo_sensor_speed1:{sumo_sensor_speed1}, vmax_sensor1:{vmax_sensor1}')

    if valor['priority'] == 4 and sumo_sensor_speed1 > vmax_sensor1:
        rotas_det[chave]['demand_num'] = np.ceil(demanda_atrib +
                                                 ((abs(np.floor(valor['sumo_sensor_speed1']) - np.ceil(valor['vmax_sensor1'])) +
                                                   abs(np.floor(valor['sumo_sensor_speed2']) - np.ceil(valor['vmax_sensor2'])))*10))
        print(
            f'prioridade4: {demanda_atrib} -> {rotas_det[chave]["demand_num"]}')

    if valor['priority'] == 3 and sumo_sensor_speed1 > vmax_sensor1:
        rotas_det[chave]['demand_num'] = np.ceil(demanda_atrib +
                                                 ((abs(np.floor(valor['sumo_sensor_speed1']) - np.ceil(valor['vmax_sensor1'])) +
                                                   abs(np.floor(valor['sumo_sensor_speed2']) - np.ceil(valor['vmax_sensor2'])))*3))
        print(
            f'prioridade3: {demanda_atrib} -> {rotas_det[chave]["demand_num"]}')

    if valor['priority'] == 1 and sumo_sensor_speed1 > vmax_sensor1:
        rotas_det[chave]['demand_num'] = np.ceil(demanda_atrib +
                                                 ((abs(np.floor(valor['sumo_sensor_speed1']) - np.ceil(valor['vmax_sensor1'])) +
                                                   abs(np.floor(valor['sumo_sensor_speed2']) - np.ceil(valor['vmax_sensor2'])))*2))
        print(
            f'prioridade4: {demanda_atrib} -> {rotas_det[chave]["demand_num"]}')


## Salvando novo xml (teste 1)

dem_xml

In [ ]:
for ln in b2tf_add:
    ln.attrib['file'] = f'resultados/detectores/{arquivo_add}.xml'

tree_add.write("adicionais.xml")

In [ ]:
tree = ET.parse('demanda.rou.xml')
root = tree.getroot()

In [ ]:
# localizar uma rota

#b2tf = root.find("./flow[@id='flow_6']")
#print(b2tf.attrib)
#b2tf.attrib['number'] = '100'
#print(b2tf.attrib)

In [ ]:
for rota in rotas_det:
    b2tf = root.find(f'./flow[@route="{rota}"]')
    b2tf.attrib['number'] = str(int(rotas_det[rota]['demand_num']))
    print(b2tf.attrib)

In [ ]:
tree.write("demanda.rou.xml")
tree.write(f'resultados/demandas/{arquivo_add}.rou.xml')

## rodar próx.passo

In [ ]:
# rodar simulação
!sumo simulacao.sumocfg


In [ ]:
# to csv
!python3 xml2csv.py full.output.xml
